In [8]:
import numpy as np
import gymnasium as gym
import random

In [9]:
class PrioritizedReplayBuffer:
    def __init__(self, capacity, alpha=0.6):
        """
        Inicializa o replay buffer com prioridades.
        :param capacity: Capacidade máxima do buffer.
        :param alpha: Parâmetro de prioridade (0 < alpha <= 1).
        """
        self.capacity = capacity
        self.alpha = alpha
        self.buffer = []
        self.priorities = np.zeros(capacity, dtype=np.float32)
        self.pos = 0

    def add(self, transition):
        """
        Adiciona uma transição ao buffer.
        :param transition: Tupla (state, action, reward, next_state, done).
        """
        max_priority = self.priorities.max() if self.buffer else 1.0
        if len(self.buffer) < self.capacity:
            self.buffer.append(transition)
        else:
            self.buffer[self.pos] = transition
        self.priorities[self.pos] = max_priority
        self.pos = (self.pos + 1) % self.capacity

    def sample(self, batch_size, beta=0.4):
        """
        Amostra um batch de transições com base nas prioridades.
        :param batch_size: Tamanho do batch.
        :param beta: Parâmetro de compensação (0 < beta <= 1).
        :return: Batch de transições, índices e pesos.
        """
        if len(self.buffer) == self.capacity:
            priorities = self.priorities
        else:
            priorities = self.priorities[:self.pos]
        probs = priorities ** self.alpha
        probs /= probs.sum()

        indices = np.random.choice(len(self.buffer), batch_size, p=probs)
        samples = [self.buffer[idx] for idx in indices]

        weights = (len(self.buffer) * probs[indices]) ** (-beta)
        weights /= weights.max()
        return samples, indices, np.array(weights, dtype=np.float32)

    def update_priorities(self, indices, priorities):
        """
        Atualiza as prioridades das transições amostradas.
        :param indices: Índices das transições.
        :param priorities: Novas prioridades.
        """
        for idx, priority in zip(indices, priorities):
            self.priorities[idx] = priority

In [10]:
def generate_dataset_with_sarsa(env, buffer_capacity=10000, num_episodes=5000, alpha=0.1, gamma=0.99, epsilon=1.0, epsilon_decay=0.995, epsilon_min=0.1):
    """
    Gera um dataset usando o algoritmo SARSA.
    :param env: Ambiente Gymnasium.
    :param buffer_capacity: Capacidade do replay buffer.
    :param num_episodes: Número de episódios para coleta de dados.
    :param alpha: Taxa de aprendizado do SARSA.
    :param gamma: Fator de desconto.
    :param epsilon: Taxa de exploração inicial.
    :param epsilon_decay: Decaimento da taxa de exploração.
    :param epsilon_min: Taxa de exploração mínima.
    :return: Replay buffer preenchido.
    """
    replay_buffer = PrioritizedReplayBuffer(buffer_capacity)

    # Inicializar a tabela Q
    num_states = env.observation_space.n
    num_actions = env.action_space.n
    q_table = np.zeros((num_states, num_actions))

    for episode in range(num_episodes):
        state, _ = env.reset()
        done = False

        # Escolher a primeira ação (epsilon-greedy)
        if np.random.rand() < epsilon:
            action = env.action_space.sample()  # Exploração
        else:
            action = np.argmax(q_table[state])  # Explotação

        episode_transitions = []  # Armazenar transições do episódio atual

        while not done:
            # Executar a ação e observar o próximo estado e recompensa
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated

            # Escolher a próxima ação (epsilon-greedy)
            if np.random.rand() < epsilon:
                next_action = env.action_space.sample()  # Exploração
            else:
                next_action = np.argmax(q_table[next_state])  # Explotação

            # Criar a transição
            transition = (state, action, reward, next_state, done)
            episode_transitions.append(transition)

            # Atualizar a tabela Q usando SARSA
            td_target = reward + gamma * q_table[next_state][next_action] * (not done)
            td_error = td_target - q_table[state][action]
            q_table[state][action] += alpha * td_error

            # Atualizar o estado e a ação
            state = next_state
            action = next_action

        # Adicionar todas as transições do episódio ao replay buffer
        for transition in episode_transitions:
            replay_buffer.add(transition)

        # Decaimento de epsilon
        epsilon = max(epsilon_min, epsilon * epsilon_decay)

        # Verificar se o episódio terminou com sucesso (entrega do passageiro)
        if terminated and reward == 20:  # Recompensa de +20 indica sucesso
            print(f"Episódio {episode + 1}: Passageiro entregue com sucesso!")

    return replay_buffer, q_table

In [11]:
def validate_dataset(replay_buffer):
    """
    Valida o dataset gerado.
    :param replay_buffer: Replay buffer contendo as transições.
    """
    # Verificar o tamanho do dataset
    print(f"Tamanho do dataset: {len(replay_buffer.buffer)}")

    # Verificar a diversidade de estados e ações
    states = [t[0] for t in replay_buffer.buffer]
    actions = [t[1] for t in replay_buffer.buffer]
    rewards = [t[2] for t in replay_buffer.buffer]

    print(f"Estados únicos: {len(np.unique(states))}")
    print(f"Ações únicas: {len(np.unique(actions))}")
    print(f"Recompensas únicas: {len(np.unique(rewards))}")

    # Verificar se há transições terminais
    terminal_transitions = [t for t in replay_buffer.buffer if t[4]]  # done=True
    print(f"Transições terminais: {len(terminal_transitions)}")

    # Verificar a distribuição de recompensas
    print(f"Recompensa mínima: {np.min(rewards)}")
    print(f"Recompensa máxima: {np.max(rewards)}")
    print(f"Recompensa média: {np.mean(rewards)}")

In [12]:
# Configuração do ambiente
env = gym.make("Taxi-v3")

# Gerar o dataset com SARSA
replay_buffer, q_table = generate_dataset_with_sarsa(env, buffer_capacity=1000000, num_episodes=10000)


Episódio 1: Passageiro entregue com sucesso!
Episódio 2: Passageiro entregue com sucesso!
Episódio 3: Passageiro entregue com sucesso!
Episódio 4: Passageiro entregue com sucesso!
Episódio 5: Passageiro entregue com sucesso!
Episódio 6: Passageiro entregue com sucesso!
Episódio 7: Passageiro entregue com sucesso!
Episódio 8: Passageiro entregue com sucesso!
Episódio 9: Passageiro entregue com sucesso!
Episódio 10: Passageiro entregue com sucesso!
Episódio 11: Passageiro entregue com sucesso!
Episódio 12: Passageiro entregue com sucesso!
Episódio 13: Passageiro entregue com sucesso!
Episódio 14: Passageiro entregue com sucesso!
Episódio 15: Passageiro entregue com sucesso!
Episódio 16: Passageiro entregue com sucesso!
Episódio 17: Passageiro entregue com sucesso!
Episódio 18: Passageiro entregue com sucesso!
Episódio 19: Passageiro entregue com sucesso!
Episódio 20: Passageiro entregue com sucesso!
Episódio 21: Passageiro entregue com sucesso!
Episódio 22: Passageiro entregue com sucess

In [13]:
# Validar o dataset
validate_dataset(replay_buffer)

Tamanho do dataset: 276711
Estados únicos: 400
Ações únicas: 6
Recompensas únicas: 3
Transições terminais: 10000
Recompensa mínima: -10
Recompensa máxima: 20
Recompensa média: -1.3438424927089996


In [14]:
# Salvar o dataset
dataset_filename = 'dataSet/DataSet_taxi.npy'
np.save(dataset_filename, np.array((env, replay_buffer.buffer), dtype=object))
print(f"Replay buffer salvo em {dataset_filename}")

Replay buffer salvo em dataSet/DataSet_taxi.npy
